### Kriptonite

A NB to drive Kryptonite.

Kryptonite reads a PMAPS file, selects events with 1 S1 and a variable number of S2s (decided by configuration) and then writesa krypton-like DST.

"Krypton-like" means literally that the events are assumed to be point-like, and thus each event can be characterized by its position and energ (x,y,z,E). The Kryptonite DST takes the following fields:

        run          = run number
        event        = absolute event_number
        timestamp    = time stamp in ms
        peak         = peak number (counts the number of S2 in the event)
        s1_energy    = energy of S1 in PES
        s2_energy    = energy of S2 in PES (corresponding to peak number)
        s1_epeak     = energy of the S1 peak in PES (max of the energy for S1)
        s2_epeak     = energy of the S2 peak in PES (max of the energy for S2)
        s1_width     = width of S1 (in time units)
        s2_width     = width of S2 (in time units)
        n_s1_pmt     = number of S1 found in the PMTs that match the S1 found in the sum
        n_sipm       = number of SiPM found in the event
        x            = x coordinate (cartesian system)
        y            = y coordinate (cartesian system)
        z            = z coordinate (cartesian system)
        r            = radial coordinate (cylindrical system)
        phi          = azimuth coordinate (cylindrical system)

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import time
import tables as tb
import numpy as np
import pandas as pd
import operator
import functools
import copy
import glob
import math
import textwrap
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
from   invisible_cities.core.system_of_units_c import units
import invisible_cities.core.fit_functions as fitf

In [ ]:
import hst_functions as hst

### Useful functions

In [ ]:
plt.rcParams["figure.figsize"]          = 6, 6
plt.rcParams["font.size"]               = 15
plt.rcParams["figure.max_open_warning"] = 100

profOpt = "--k"
fitOpt  = "r"

In [ ]:
pi = np.pi

### run number

In [ ]:
run_number=3389

### Files

In [ ]:
input_file = os.path.join(os.environ['IC_DATA'],
                        'LSC/pmaps/{}/kdst_{}_full.h5'.format(run_number, run_number))

In [ ]:
input_file = os.path.expandvars("$IC_DATA/kdst_3389_full.h5")

In [ ]:
input_file

### data selection

{'n_tot': 489000, 's1_not_1': 178624, 's1_eq_1': 310284, 's2_eq_s2_mult': 309200, 'not_si': 3030, 's2_not_eq_s2_mult': 1084, 'not_s1': 92})

### Functions

In [ ]:
def read_kdst(kdst_file_name):
    """Return the kdst as PD DataFrames."""
    with tb.open_file(kdst_file_name, 'r') as h5f:
        t   = h5f.root.KrEvent.KrEvent
        
        return (pd.DataFrame.from_records(t.read()))

In [ ]:
def trigger_rate(timestamp, event):
    times = timestamp * 1e-3
    t0 = np.min(times)
    t1 = np.max(times)
    e0 = np.min(event)
    e1 = np.max(event) 
    print("First event = {} t = {} s".format(e0,t0))
    print("Last event = {} t = {} s".format(e1,t1))
    print("Dt = {}  s".format(t1-t0))
    print("Trigger rate: {} evts/s".format((e1-e0)/(t1-t0)))

### Read DST

In [ ]:
kdst = read_kdst(input_file)

In [ ]:
kdst.head()

In [ ]:
trigger_rate(kdst.timestamp.values, kdst.event.values)

In [ ]:
def histo2d(x,y, xrange, yrange, xlabel, ylabel, nbin=(50, 50), log=False):
    if log:
        hst.hist2d(x, y, nbin, range=(xrange, yrange), norm=LogNorm())
    else:
        hst.hist2d(x, y, nbin, range=(xrange, yrange))
    hst.labels(xlabel, ylabel)

In [ ]:
def histo_event_rate(timestamp, event):
    """Compute event rate"""
    times = timestamp * 1e-3
    T    = times 
    t0 = np.min(times)
    t1 = np.max(times)
    e0 = np.min(event)
    e1 = np.max(event) 
    dt = (e1 - e0)/(t1-t0)
    r = (T - t0)
    hst.hist(r,100)
    hst.labels("Time (sec)", "Event rate (({} sec)$^{{-1}}$)".format(dt))
    #save("EventRate")

    

In [ ]:
histo_event_rate(kdst.timestamp.values, kdst.event.values)

### S1 histos

In [ ]:
hst.hist(kdst.s1_energy, 100, (0, 50))
hst.labels("S1 energy in pes", "# of events")

In [ ]:
hst.hist(kdst.s1_epeak, 100, (0, 10))
hst.labels("S1 epeak in pes", "# of events")

In [ ]:
hst.hist(kdst.s1_tpeak.values/units.mus, 100, (0, 650))
hst.labels("S1 time in $\mu$s", "# of events")

In [ ]:
hst.hist(kdst.s1_width.values/units.ns, 100, (0, 500))
hst.labels("S1 width in ns","# of events")

### S2 histos

In [ ]:
hst.hist(kdst.s2_energy, 100, (0, 10000))
hst.labels("S2 energy in pes", "# of events")

In [ ]:
hst.hist(kdst.s2_epeak, 100, (500, 5000))
hst.labels("S2 peak in pes", "# of events")

In [ ]:
hst.hist(kdst.s2_tpeak.values/units.mus, 20, (600, 700))
hst.labels("S2 time in $\mu$s", "# of events")

In [ ]:
hst.hist(kdst.s2_width.values/units.mus, 100, (0, 20))
hst.labels("S1 width in $\mu$s","# of events")

### event histos

In [ ]:
def drift_time(ts1, ts2):
    return ts2 - ts1

In [ ]:
dt = drift_time(kdst.s1_tpeak.values/units.mus, kdst.s2_tpeak.values/units.mus)

In [ ]:
def histo_drift_time(ts1, ts2):
    dt = drift_time(ts1, ts2)
    hst.hist(dt, 100, (0, 600))
    hst.labels("drift time in $\mu$s", "# of events")

In [ ]:
histo_drift_time(kdst.s1_tpeak.values/units.mus, kdst.s2_tpeak.values/units.mus)

In [ ]:
hst.hist(kdst.z/units.mm, 100, (0, 600))
hst.labels("z mm", "# of events")

In [ ]:
hst.hist(kdst.n_sipm, 20, (0, 10))
hst.labels("# sipms with signal", "# of events")

In [ ]:
hst.hist(kdst.Q_tp, 100, (0, 500))
hst.labels("# charge in tracking plane", "# of events")

In [ ]:
hst.hist(kdst.x, 100, (-220, 220))
hst.labels("x (mm)", "# of events")

In [ ]:
hst.hist(kdst.y, 100, (-220, 220))
hst.labels("y (mm)", "# of events")

In [ ]:
hst.hist(kdst.r, 50, (0, 220))
hst.labels(" r (mm)", "# of events")

In [ ]:
hst.hist(kdst.phi, 50, (-pi, pi))
hst.labels(" phi (rad)", "# of events")

In [ ]:
zrange = (0, 200)
erange = (4e3, 9e3)
histo2d(kdst.z, kdst.s2_energy, zrange, erange,'z (mm)','S2 energy (pes)')

In [ ]:
histo2d(dt, kdst.s2_energy, (0, 200), (4e3, 10e3),"dt ($\mu$s)","S2 energy PES")

In [ ]:
histo2d(kdst.x, kdst.y, (-215,215), (-215,215), 'x (mm)', 'y (mm)', log=False)

### Fiducial cut at R < 100, z < 220 (mm)

In [ ]:
rfid   = 100
zfid   = 200
xrange = (-rfid, rfid)
yrange = (-rfid, rfid)
zrange = (    0, zfid)
rrange = (    0, rfid)
prange = (  -pi,   pi)

In [ ]:
def fid_cut(kdst, r_cut=rrange[1], z_cut=zrange[1]):
    kr = kdst.loc[kdst['r'] <= r_cut]
    kz = kr.loc[kdst['z'] <= z_cut]
    
    return kz

In [ ]:
kfid = fid_cut(kdst, r_cut=rfid, z_cut=zfid)

In [ ]:
kfid.head()

In [ ]:
histo2d(kfid.x, kfid.y, (-215,215), (-215,215), 'x (mm)', 'y (mm)', log=False)

In [ ]:
hst.hist(kfid.z/units.mm, 100, (0, 600))
hst.labels("z mm", "# of events")

### Fit to S2 vs Z distribution

In [ ]:
def fit_xy_profile_to_exp(x, y, xrange, yrange, fit_bins, xfit, xlabel, ylabel, fit_par):
    """Fit the xy distribution to an expo and return correction function"""
    
    histo2d(x, y, xrange, yrange, xlabel, ylabel)
    x, y, _ = fitf.profileX(x, y, fit_bins, xrange, yrange)
    plt.plot(x, y, profOpt)

    sel  = fitf.in_range(x, xfit[0], xfit[1])
    x, y = x[sel], y[sel]

    f = fitf.fit(fitf.expo, x, y, fit_par)
    plt.plot(x, f.fn(x), fitOpt)
    print(f.values)

    zfun  = f.fn
    zmin  = np.min(x)
    zcorr = lambda z: zfun(zmin)/zfun(z)
    return zcorr
    

In [ ]:
zcorr = fit_xy_profile_to_exp(kfid.z, kfid.s2_energy, 
                              xrange   = zrange, 
                              yrange   = (4e3, 9.0e3), 
                              fit_bins = 50, 
                              xfit     = (50, 130), 
                              xlabel   = 'z (mm)',
                              ylabel   = 'S2 energy (pes)', 
                              fit_par  = (1e4, -200))

In [ ]:
z = np.arange(0, 200)
plt.plot(z, zcorr(z))

In [ ]:
def energy_correct_by_attachment(z, e, zcorr, erange, fitrange, fit_par):
    ecorr = e * zcorr(z)
    y, x, _ = hst.hist(ecorr, 100, range=erange)
    x       = x[:-1] + np.diff(x) * 0.5
    sel     = fitf.in_range(x, fitrange[0], fitrange[1])
    x, y    = x[sel], y[sel]
    f       = fitf.fit(fitf.gauss, x, y, fit_par)
    plt.plot(x, f.fn(x), fitOpt)
    plt.text(erange[0], max(y)/2, hst.gausstext(f.values))

    hst.labels("S2 energy (pes)", "Entries")

In [ ]:
energy_correct_by_attachment(kfid.z, kfid.s2_energy, zcorr, 
                             erange   = ( 5e3, 11e3),
                             fitrange = (7500, 9100),
                             fit_par  = (1e3, 7e3, 200))

In [ ]:
def e_z_corr(E, Z, zcorr):
    return E * zcorr(Z)
    

In [ ]:
Ez = e_z_corr(kfid.s2_energy, kfid.z, zcorr)

In [ ]:
zcorr2 = fit_xy_profile_to_exp(kfid.z, 
                               e_z_corr(kfid.s2_energy, kfid.z, zcorr), 
                               xrange   = (50, 150),
                               yrange   = (4e3, 9.0e3), 
                               fit_bins = 50, 
                               xfit     = (70, 120),
                               xlabel   = 'z (mm)',
                               ylabel   = 'S2 z corr (pes)', 
                               fit_par  = (1e4, -200))

In [ ]:
z = np.arange(0, 200)
plt.plot(z, zcorr2(z))

In [ ]:
energy_correct_by_attachment(kfid.z, 
                             Ez, 
                             zcorr2, 
                             erange   = ( 5e3, 11e3),
                             fitrange = (7500, 9100),
                             fit_par  = (1e3, 7e3, 200))

### Geometrical corrections

#### At least one SiPM with signal

In [ ]:
def nsipm_cut(kdst, n_cut=1):
    return kdst.loc[kdst['n_sipm'] > n_cut]

In [ ]:
ksipm = nsipm_cut(kdst, n_cut=1)

In [ ]:
hst.hist(ksipm.Q_tp, 100, (0, 500))
hst.labels("# charge in tracking plane", "# of events")

#### Energy range

In [ ]:
Erange =  (1e3, 1e4)

In [ ]:
def energy_cut(kdst, Emin=4000, Emax=9000):
    kmin = kdst.loc[kdst['s2_energy'] > Emin]
    kmax = kmin.loc[kmin['s2_energy'] < Emax]
    return kmax

### XY corrections

In [ ]:
kene = energy_cut(ksipm, Emin=Erange[0], Emax=Erange[1])

In [ ]:
kenefid = kene.loc[kene.r < 180]

In [ ]:
hst.hist2d(kenefid.r,
           kenefid.s2_energy * zcorr(kenefid.z),
           (50, 50), range=((0, 215),Erange))
cb.set_label("E (pes)")
hst.labels("x (mm)", "y (mm)")

In [ ]:
pf, _, cb = hst.profile_and_scatter(kenefid.x.values, kenefid.y.values,
                                    kenefid.s2_energy.values * zcorr(kenefid.z.values),
                                    (50, 50), xrange=(-215, 215), yrange=(-215, 215))
cb.set_label("E (pes)")
hst.labels("x (mm)", "y (mm)")

In [ ]:
def get_xycorr(xs, ys, es, eref=None):
    if eref is None:
        eref = es[es.shape[0]//2, es.shape[1]//2]
    xs   = xs.reshape(xs.size, 1)
    ys   = ys.reshape(ys.size, 1)
    print(eref)
    def xycorr(x, y):
        x_closest = np.apply_along_axis(np.argmin, 0, abs(x-xs))
        y_closest = np.apply_along_axis(np.argmin, 0, abs(y-ys))
        e         = es[x_closest, y_closest]
        e[e<1e3]  = eref
        return eref/ e
    return xycorr
xycorr = get_xycorr(*pf[:-1])

### xyz corrections

In [ ]:
def e_xy_corr(E, X, Y):
    return E * xycorr(X, Y)

def e_xyz_corr(E, X, Y, Z):
    return E * zcorr(Z) * xycorr(X, Y)

In [ ]:
E  = e_xyz_corr(kenefid.s2_energy.values, kenefid.x.values, kenefid.y.values, kenefid.z.values)
y, x, _ = \
hst.hist(E, 100, Erange)

x = x[:-1] + np.diff(x) * 0.5
f = fitf.fit(fitf.gauss, x, y, (1e5, 1e4, 1e2))
plt.plot(x, f.fn(x), fitOpt)
plt.text(Erange[0], max(y)/2, hst.gausstext(f.values))

hst.labels("E (pes)", "Entries")

In [ ]:
pf, _, cb = hst.profile_and_scatter(kenefid.x.values, kenefid.y.values, E,
                                    (50, 50), xrange=(-215, 215), yrange=(-215, 215))
cb.set_label("E (pes)")
hst.labels("x (mm)", "y (mm)")
#save("EvsXY")

In [ ]:
nevts, xs, ys, _  = hst.hist2d(kene.x.values, kene.y.values, (50, 50), range=((-215, 215), (-215, 215)))
hst.labels("x (mm)", "y (mm)")